In [106]:
import pandas as pd
import numpy as np

from scipy import stats
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

from scipy.stats import chi2
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split

In [107]:
# Lettura del dataset iniziale già processato
campaign_data = pd.read_csv('../data/campaign-data-processed.csv')

#### One-hot encoding

In [108]:
# One-hot encoding per "os_id"
one_hot_encoded = pd.get_dummies(campaign_data['os_id'])
campaign_data = pd.concat([campaign_data, one_hot_encoded], axis = 1)
campaign_data.drop('os_id', axis = 1, inplace = True)

# One-hot encoding per "browser_id"
one_hot_encoded = pd.get_dummies(campaign_data['browser_id'])
campaign_data = pd.concat([campaign_data, one_hot_encoded], axis = 1)
campaign_data.drop('browser_id', axis = 1, inplace = True)

# One-hot encoding per "device_type_id"
one_hot_encoded = pd.get_dummies(campaign_data['device_type_id'])
campaign_data = pd.concat([campaign_data, one_hot_encoded], axis = 1)
campaign_data.drop('device_type_id', axis = 1, inplace = True)

#### Assegnazione della label "clicker" (classification)

In [109]:
# Funzione che crea la label "clicker" per la riga in input
def assignLabel(row):
    
    if(row['clicks'] >= 1): 
        return 1
    else: 
        return 0

# Creazione della label "clicker" nel dataset principale
campaign_data['clicker'] = campaign_data.apply(lambda row: assignLabel(row), axis = 1)

#### Calcolo dello score (regression)

In [110]:
pos_class_data = campaign_data[campaign_data['clicker'] == 1].copy()
neg_class_data = campaign_data[campaign_data['clicker'] == 0].copy()

pos_class_data['score'] = round(pos_class_data['clicks'] / pos_class_data['total_impressions'], 3)
neg_class_data['score'] = - round((neg_class_data['total_impressions'] - 1) / neg_class_data['total_impressions'], 3)

campaign_data = pd.concat([neg_class_data, pos_class_data])
scores = campaign_data['score']

campaign_data.drop(['clicks', 'score'], inplace = True, axis = 1)

#### Suddivisione in training e test set

In [111]:
# Splitting in training e testing set (classification)
campaign_data, testing_set = train_test_split(campaign_data, train_size = 0.75, stratify = campaign_data['clicker'], random_state = 19)
testing_set.to_csv('../data/campaign-data-testing.csv', index = False)

In [112]:
# Testing set regression
testing_set['score'] = scores
testing_set.drop('clicker', inplace = True, axis = 1)
testing_set.to_csv('../data/campaign-data-testing-regression.csv', index = False)

#### Eliminazione dei duplicati

In [113]:
# Rimozione dei duplicati (maggiore affidabilità e no introduzione di bias)
campaign_data = campaign_data.drop_duplicates()

#### Feature selection

In [114]:
"""
"""
def featureSelection(campaign_data):

    # Calcolo della matrice di correlazione tra tutte le variabili del dataframe
    corr_matrix = campaign_data.corr()

    # Rilevamento delle feature con elevata correlazione positiva o negativa (>= 0.7)
    high_corr_feature = corr_matrix[(abs(corr_matrix) >= 0.7) & (abs(corr_matrix) < 1)]
    high_corr_feature = high_corr_feature.index[high_corr_feature.any(axis = 1)].tolist()

    # Analisi della multicollinearity tra le feature rilevate correlate, utilizzando il VIF
    multicollinear_feature = []

    # Ciclo che computa il VIF e rimuove una alla votla le variabili problematiche (VIF >= 5)
    while(True):

        # Computazione del VIF
        vif_data = pd.DataFrame(data = high_corr_feature, columns = ['feature'])
        vif_data['VIF'] = [ variance_inflation_factor(campaign_data[high_corr_feature].values, i) for i in range(len(high_corr_feature)) ]
        vif_data = vif_data.sort_values('VIF', ascending = False)

        # VIF con valore più elevato
        highest_vif = vif_data['VIF'].iloc[0]

        # Se tutte le feature hanno VIF < 5, allora conclude
        if(highest_vif < 5):
            break
        
        # Feature con valore del VIF più alto
        highest_vif_feature = vif_data['feature'].iloc[0]
        multicollinear_feature.append(highest_vif_feature)
        high_corr_feature.remove(highest_vif_feature)

    # Eliminazione delle feature problematiche
    campaign_data_filtered = campaign_data.drop(multicollinear_feature, axis = 1)

    return campaign_data_filtered

In [115]:
# Feature selection sull'insieme totale delle osservazioni
campaign_data = featureSelection(campaign_data)

#### Rimozione degli outliers

In [116]:
""" 
Funzione che calcola la distanza di Mahalanobis tra ogni riga del dataframe X in input ed il dataframe data (distribuzione) 

X -> dataframe/osservazioni per le quali calcolare la distanza di Mahalanobis dalla distribuzione
data -> dataframe con la distribuzione di partenza
cov -> covariance matrix della distribuzione di partenza (if None, viene computata direttamente sulla distribuzione iniziale in input)

return -> la distanza di Mahalanobis delle osservazioni di X rispetto alla distribuzione data
"""
def computeMahalanobis(X = None, data = None, cov = None):
    
    # Calcolo della covariance matrix (regularization technique to handle singular covariance matrix) -> regularized Mahalanobis distance
    if(not cov):
        cov = np.cov(data.values.T) + 0.01 * np.eye(len(data.columns))
    
    # Mahalanobis Distance formula 
    return np.round(np.dot((np.dot((X - np.mean(data)), np.linalg.inv(cov))), (X - np.mean(data)).T).diagonal(), 3)

In [117]:
# Standardization (utile per test chi-quadro)
clicker = campaign_data['clicker']
campaign_data_standardized = campaign_data.drop('clicker', axis = 1).apply(lambda col: StandardScaler().fit_transform(col.values.reshape(-1,1)).flatten())
campaign_data_standardized['clicker'] = clicker

In [118]:
# Classe positiva -> clicker = 1
pos_class_data = campaign_data_standardized[campaign_data_standardized['clicker'] == 1].copy()

# Calcolo della distanza di Mahalanobis per ciascuna riga del dataframe rispetto alla distribuzione totale
pos_class_data['mahal'] = computeMahalanobis(X = pos_class_data, data = pos_class_data)

# Setting del threshold con test chi-quadro (confidenza 0.95)
threshold = chi2.ppf((1 - 0.001), df = len(pos_class_data.columns) - 1)

# Eliminazione degli outliers in base al threshold stabilito (superato il limite massimo di distanza dalla distribuzione)
pos_class_data = pos_class_data[pos_class_data['mahal'] < threshold]

# Filtraggio del dataframe non standardizzato
pos_class_data = campaign_data.loc[pos_class_data.index]

In [119]:
# Classe negativa -> clicker = 0
neg_class_data = campaign_data_standardized[campaign_data_standardized['clicker'] == 0].copy()

# Calcolo della distanza di Mahalanobis per ciascuna riga del dataframe rispetto alla distribuzione totale
neg_class_data['mahal'] = computeMahalanobis(X = neg_class_data, data = neg_class_data)

# Setting del threshold con test chi-quadro (confidenza 0.95)
threshold = chi2.ppf((1 - 0.1), df = len(neg_class_data.columns) - 1)

# Eliminazione degli outliers in base al threshold stabilito (superato il limite massimo di distanza dalla distribuzione)
neg_class_data = neg_class_data[neg_class_data['mahal'] < threshold]

# Filtraggio del dataframe non standardizzato
neg_class_data = campaign_data.loc[neg_class_data.index]

In [120]:
# Dataset globale campaign senza outliers e post feature selection
campaign_data = pd.concat([neg_class_data, pos_class_data])

#### Dataset finale (training)

In [121]:
# Scrittura del file CSV con training set (classification)
campaign_data.to_csv('../data/campaign-data-training.csv', index = False)

In [122]:
campaign_data['score'] = scores
campaign_data.drop(['clicker', 'total_impressions'], axis = 1, inplace = True)

# Scrittura del file CSV con training set (regression)
campaign_data.to_csv('../data/campaign-data-training-regression.csv', index = False)